# Mini Pytato 

In [1]:
import numpy as np
import numpy.linalg as la
import pymbolic.primitives as p
import loopy as lp
import pyopencl as cl
import pyopencl.array

ctx = cl.create_some_context(interactive=True)
queue = cl.CommandQueue(ctx)

Choose platform:
[0] <pyopencl.Platform 'Intel(R) OpenCL HD Graphics' at 0x4270220>
[1] <pyopencl.Platform 'Portable Computing Language' at 0x7fe6b28e08e8>


Choice [0]: 1


Set the environment variable PYOPENCL_CTX='1' to avoid being asked again.


In [2]:
class Array:
    def __init__(self):
        self.shape = (10, 10)
        self.dtype = np.float64
        
    def __add__(self, other):
        return Sum(self, other)
    
    def __mul__(self, other):
        return Product(self, other)
        
class Sum(Array):
    def __init__(self, a, b):
        super().__init__()
        self.a = a
        self.b = b
        
    mapper_method = "map_sum"
        
class Product(Array):
    def __init__(self, a, b):
        super().__init__()
        self.a = a
        self.b = b
        
    mapper_method = "map_product"
        
class Placeholder(Array):
    def __init__(self, name):
        super().__init__()
        self.name = name
        
    mapper_method = "map_placeholder"

In [3]:
class Mapper:
    def rec(self, ary):
        return getattr(self, ary.mapper_method)(ary)

In [4]:
class CodegenMapper(Mapper):
    def map_sum(self, expr):
        return self.rec(expr.a) + self.rec(expr.b)
    
    def map_product(self, expr):
        return self.rec(expr.a) * self.rec(expr.b)
    
    def map_placeholder(self, expr):
        return p.Variable(expr.name)[p.Variable("i"), p.Variable("j")]

In [5]:
x = Placeholder("x")
y = Placeholder("y")

expr = (x+x*y)*x

# expr = (x+y)
# expr = expr*expr
# expr = expr*expr
# expr = expr*expr
# expr = expr*expr
# expr = expr*expr

In [6]:
print(CodegenMapper().rec(expr))

(x[i, j] + x[i, j]*y[i, j])*x[i, j]


In [7]:
knl = lp.make_kernel(
    "{[i,j]: 0<=i,j<10}",
    [lp.Assignment(
        p.Variable("lhs")[p.Variable("i"), p.Variable("j")], 
        CodegenMapper().rec(expr)
    )])
print(knl)

---------------------------------------------------------------------------
KERNEL: loopy_kernel
---------------------------------------------------------------------------
ARGUMENTS:
lhs: type: <auto/runtime>, shape: (10, 10), dim_tags: (N1:stride:10, N0:stride:1) out aspace: global
x: type: <auto/runtime>, shape: (10, 10), dim_tags: (N1:stride:10, N0:stride:1) in aspace: global
y: type: <auto/runtime>, shape: (10, 10), dim_tags: (N1:stride:10, N0:stride:1) in aspace: global
---------------------------------------------------------------------------
DOMAINS:
{ [i, j] : 0 <= i <= 9 and 0 <= j <= 9 }
---------------------------------------------------------------------------
INAME TAGS:
i: None
j: None
---------------------------------------------------------------------------
INSTRUCTIONS:
for i, j
    lhs[i, j] = (x[i, j] + x[i, j]*y[i, j])*x[i, j]  {id=insn}
end i, j
---------------------------------------------------------------------------


/home/andreas/src/loopy/loopy/kernel/creation.py:2595: LoopyWarning: 'lang_version' was not passed to make_function(). To avoid this warning, pass lang_version=(2018, 2) in this invocation. (Or say 'from loopy.version import LOOPY_USE_LANGUAGE_VERSION_2018_2' in the global scope of the calling frame.)
  tunit = make_function(*args, **kwargs)


In [8]:
xval = np.random.randn(10, 10)
yval = np.random.randn(10, 10)

evt, (res,) = knl(queue, x=xval, y=yval)

/tmp/ipykernel_301600/976168068.py:4: DirectCallUncachedWarning: TranslationUnit.__call__ will become uncached in 2024, meaning it will incur possibly substantial compilation cost with every invocation. Use TranslationUnit.executor to obtain an object that holds longer-lived caches.
  evt, (res,) = knl(queue, x=xval, y=yval)


In [9]:
print(la.norm(res- (xval+xval*yval)*xval))

3.818293849341007e-15


In [10]:
knl = lp.add_and_infer_dtypes(knl, {"x": xval.dtype, "y": yval.dtype})

code = lp.generate_code_v2(knl).device_code()
print(code)

#define lid(N) ((int) get_local_id(N))
#define gid(N) ((int) get_group_id(N))
#if __OPENCL_C_VERSION__ < 120
#pragma OPENCL EXTENSION cl_khr_fp64: enable
#endif

__kernel void __attribute__ ((reqd_work_group_size(1, 1, 1))) loopy_kernel(__global double *__restrict__ lhs, __global double const *__restrict__ x, __global double const *__restrict__ y)
{
  for (int j = 0; j <= 9; ++j)
    for (int i = 0; i <= 9; ++i)
      lhs[10 * i + j] = (x[10 * i + j] + x[10 * i + j] * y[10 * i + j]) * x[10 * i + j];
}


In [11]:
prg = cl.Program(ctx, code).build()

In [12]:
xval_dev = cl.array.to_device(queue, xval)
yval_dev = cl.array.to_device(queue, yval)
lhs_dev = cl.array.empty_like(xval_dev)

prg.loopy_kernel(queue, (1,), (1,), lhs_dev.data, xval_dev.data, yval_dev.data)

In [13]:
lhs = lhs_dev.get()
print(la.norm(lhs - res))

0.0
